In [ ]:
!pip install matplotlib scikit-image gunpowder
!pip install git+https://github.com/funkelab/lsd.git

In [ ]:
import h5py
import io
import matplotlib.pyplot as plt
import numpy as np
import requests
import skimage

from lsd.train import local_shape_descriptor
from skimage.filters import sobel
from skimage.measure import label
from skimage.segmentation import watershed

%matplotlib inline

In [ ]:
# get coins dataset
data = skimage.data.coins()

plt.imshow(data, cmap='gray')

In [ ]:
# create edges
edges = sobel(data)

# generate markers for watershed
markers = np.zeros_like(data)
foreground, background = 1, 2
markers[data < 30.0] = background
markers[data > 150.0] = foreground

# get unique labels
ws = watershed(edges, markers)
labels = label(ws == foreground).astype(np.uint64)

plt.imshow(data, cmap='gray')
plt.imshow(labels, cmap='prism', alpha=0.5)

In [ ]:
# calculate lsds
lsds = local_shape_descriptor.get_local_shape_descriptors(
              segmentation=labels,
              sigma=(15,)*2,
              voxel_size=(1,)*2)

print(lsds.shape)

In [ ]:
def view_lsds(lsds):

  fig, axes = plt.subplots(
            2,
            3,
            figsize=(25, 10),
            sharex=True,
            sharey=True,
            squeeze=False)
  
  axes[0][0].imshow(np.squeeze(lsds[0]), cmap='jet')
  axes[0][0].title.set_text('Mean offset Y')

  axes[0][1].imshow(np.squeeze(lsds[1]), cmap='jet')
  axes[0][1].title.set_text('Mean offset X')

  axes[0][2].imshow(np.squeeze(lsds[2]), cmap='jet')
  axes[0][2].title.set_text('Covariance Y-Y')

  axes[1][0].imshow(np.squeeze(lsds[3]), cmap='jet')
  axes[1][0].title.set_text('Covariance X-X')

  axes[1][1].imshow(np.squeeze(lsds[4]), cmap='jet')
  axes[1][1].title.set_text('Covariance Y-X')

  axes[1][2].imshow(np.squeeze(lsds[5]), cmap='jet')
  axes[1][2].title.set_text('Size')

In [ ]:
view_lsds(lsds)

In [ ]:
# example data from cremi challenge - might take a couple seconds to download
url = 'https://cremi.org/static/data/sample_A_20160501.hdf'

# convert from binary
container = h5py.File(io.BytesIO(requests.get(url).content), 'r')

In [ ]:
# get corner patch
raw = np.squeeze(container['volumes/raw'][0:1,0:250,0:250])
labels = np.squeeze(container['volumes/labels/neuron_ids'][0:1,0:250,0:250])

plt.imshow(raw, cmap='gray')
plt.imshow(labels, cmap='prism', alpha=0.5)

In [ ]:
# calc lsds
lsds = local_shape_descriptor.get_local_shape_descriptors(
              segmentation=labels,
              sigma=(100,)*2,
              voxel_size=[4,4])

In [ ]:
view_lsds(lsds)